In [ ]:
!pip install transformers pandas numpy torch scikit-learn


In [1]:
## Import Depandencies

## Import Dependencies

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler


## Step 1: Load Data

In [ ]:
# Load the dataset
df = pd.read_csv('/content/mismatch_data - Sheet1 (1).csv', encoding='utf-8')




In [ ]:

df

,Text,label,Unnamed: 2,Unnamed: 3
0,চাল্লা কি মানুহ,1,NaN,NaN
1,নাটক কৰিবলৈ স্থান নাইপোৱা চাগে.........মুঠতে চ...,1,NaN,NaN
2,আপোনালোকে বহিঃৰাইজত গৈ কি ভাষা মাতিবলগীয়া হয়...,1,NaN,NaN
3,অসমত কিমান জনে চাকৰি পাইছে।।যি কেইটামান ওলাই চ...,1,NaN,NaN
4,আমি যেতিয়া অসমীয়া মাধ্যমত শিক্ষা লৈছিলোঁ তেত...,1,NaN,NaN
...,...,...,...,...
131,।।। ঘেণ্টা!!!\n তই সাধাৰণ VCD এখনৰ নৃত্য পৰিচা...,1,NaN,NaN
132,.নাৰীয়েই মেকআপ কৰিব লৰাই তো লিপষ্টিক কাজল লগাই...,1,NaN,NaN
133,তোমাৰ মা ও হয় তো প্রসাধনৰ নামত কিবা অলপ সালনিহ...,1,NaN,NaN
134,"সিহঁত ""social servant"" হে। তাৰ বাহিৰে বেলেগ এক...",0,NaN,NaN


### data preprocessing

In [ ]:
df = df.drop(columns=['Unnamed: 2', 'Unnamed: 3'])

In [ ]:
df.head()

,Text,label
0,চাল্লা কি মানুহ,1
1,নাটক কৰিবলৈ স্থান নাইপোৱা চাগে.........মুঠতে চ...,1
2,আপোনালোকে বহিঃৰাইজত গৈ কি ভাষা মাতিবলগীয়া হয়...,1
3,অসমত কিমান জনে চাকৰি পাইছে।।যি কেইটামান ওলাই চ...,1
4,আমি যেতিয়া অসমীয়া মাধ্যমত শিক্ষা লৈছিলোঁ তেত...,1


In [ ]:
df.isnull().sum()

Text     0
label    1
dtype: int64

In [ ]:
df = df.dropna(subset=['label'])

In [ ]:
df.isnull().sum()

Text     0
label    0
dtype: int64

In [ ]:
df['label'].unique()

array(['1', '0', '-'], dtype=object)

In [ ]:
df = df[df['label'] != '-']
df

,Text,label
0,চাল্লা কি মানুহ,1
1,নাটক কৰিবলৈ স্থান নাইপোৱা চাগে.........মুঠতে চ...,1
2,আপোনালোকে বহিঃৰাইজত গৈ কি ভাষা মাতিবলগীয়া হয়...,1
3,অসমত কিমান জনে চাকৰি পাইছে।।যি কেইটামান ওলাই চ...,1
4,আমি যেতিয়া অসমীয়া মাধ্যমত শিক্ষা লৈছিলোঁ তেত...,1
...,...,...
131,।।। ঘেণ্টা!!!\n তই সাধাৰণ VCD এখনৰ নৃত্য পৰিচা...,1
132,.নাৰীয়েই মেকআপ কৰিব লৰাই তো লিপষ্টিক কাজল লগাই...,1
133,তোমাৰ মা ও হয় তো প্রসাধনৰ নামত কিবা অলপ সালনিহ...,1
134,"সিহঁত ""social servant"" হে। তাৰ বাহিৰে বেলেগ এক...",0


In [ ]:
df.isnull().sum()


Text     0
label    0
dtype: int64

In [ ]:

df['label'].unique()





array(['1', '0'], dtype=object)

In [ ]:
df.shape

(134, 2)

In [ ]:
df.dtypes

Text     object
label    object
dtype: object

In [ ]:
df['label'] = df['label'].astype(int)

In [ ]:
df.dtypes

Text     object
label     int64
dtype: object

## Tokenize and spliting the data

In [ ]:
# Tokenize using BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_length = 128

encoded_data = tokenizer.batch_encode_plus(
    df['Text'].tolist(),
    add_special_tokens=True,
    return_attention_mask=True,
    padding=True,
    truncation=True,
    max_length=max_length,
    return_tensors='pt'
)

input_ids = encoded_data['input_ids']
attention_masks = encoded_data['attention_mask']
labels = torch.tensor(df['label'].tolist())

# Split dataset into train, validation, and test sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    input_ids, labels, random_state=42, test_size=0.1)

val_texts, test_texts, val_labels, test_labels = train_test_split(
    val_texts, val_labels, random_state=42, test_size=0.5)

## Load Dataloder

In [ ]:
# Create TensorDatasets
train_dataset = TensorDataset(train_texts, train_labels)
val_dataset = TensorDataset(val_texts, val_labels)
test_dataset = TensorDataset(test_texts, test_labels)

# Create DataLoaders
batch_size = 32

train_dataloader = DataLoader(
    train_dataset,
    sampler=RandomSampler(train_dataset),
    batch_size=batch_size
)

val_dataloader = DataLoader(
    val_dataset,
    sampler=SequentialSampler(val_dataset),
    batch_size=batch_size
)

test_dataloader = DataLoader(
    test_dataset,
    sampler=SequentialSampler(test_dataset),
    batch_size=batch_size
)


## Loading the model

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=2,
    output_attentions=False,
    output_hidden_states=False
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
epochs = 4
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
for batch in train_dataloader:
    print(batch)


[tensor([[  101,  1377, 29915,  ...,     0,     0,     0],
        [  101,  1377, 29915,  ...,     0,     0,     0],
        [  101,  1347, 29912,  ...,     0,     0,     0],
        ...,
        [  101,   100,  1370,  ...,     0,     0,     0],
        [  101,   100,  1353,  ...,     0,     0,     0],
        [  101,  1370, 29885,  ...,     0,     0,     0]]), tensor([1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0,
        0, 1, 1, 1, 1, 0, 0, 0])]
[tensor([[  101,  1370, 29885,  ...,     0,     0,     0],
        [  101,   100,   100,  ...,     0,     0,     0],
        [  101,   100,  1364,  ...,     0,     0,     0],
        ...,
        [  101,  1366, 29914,  ...,     0,     0,     0],
        [  101,   100,  1354,  ...,     0,     0,     0],
        [  101,  1368, 29914,  ...,     0,     0,     0]]), tensor([0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1,
        1, 0, 0, 1, 1, 1, 1, 1])]
[tensor([[  101,  1356, 29914,  ...,     0, 

## Training The Model

In [ ]:
# Set the model to training mode
model.train()

# Loop through each epoch
for epoch in range(epochs):
    # Initialize variables for each epoch
    total_train_loss = 0

    # Loop through each batch in the training dataloader
    for batch_input_ids, batch_labels in train_dataloader:
        # Move the batch tensors to the GPU if available
        batch_input_ids = batch_input_ids.to(device)
        batch_labels = batch_labels.to(device)

        # Clear any previously calculated gradients
        model.zero_grad()

        # Perform a forward pass (evaluate the model on this training batch)
        outputs = model(input_ids=batch_input_ids, labels=batch_labels)

        # Extract loss from the outputs
        loss = outputs.loss

        # Accumulate the training loss over all batches
        total_train_loss += loss.item()

        # Perform a backward pass to calculate gradients
        loss.backward()

        # Clip the norm of the gradients to prevent the exploding gradient problem
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        # Update parameters and take a step using the optimizer
        optimizer.step()

        # Update the learning rate scheduler
        scheduler.step()

    # Calculate the average training loss for this epoch
    avg_train_loss = total_train_loss / len(train_dataloader)

    # Print the average training loss for this epoch
    print(f'Epoch {epoch + 1}/{epochs}, Average Training Loss: {avg_train_loss}')


Epoch 1/4, Average Training Loss: 0.6798426657915115
Epoch 2/4, Average Training Loss: 0.6543384939432144
Epoch 3/4, Average Training Loss: 0.6302207857370377
Epoch 4/4, Average Training Loss: 0.6298208385705948


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score


## Evaluation of the model

In [ ]:
# Set the model to evaluation mode
model.eval()

# Initialize variables for evaluation
total_eval_accuracy = 0
total_eval_loss = 0
nb_eval_steps = 0
eval_predictions = []
eval_labels = []

# Evaluate data for each batch in validation dataloader
for batch_input_ids, batch_labels in val_dataloader:
    # Move the batch tensors to the GPU if available
    batch_input_ids = batch_input_ids.to(device)
    batch_labels = batch_labels.to(device)

    # Disable gradient calculation
    with torch.no_grad():
        # Forward pass
        outputs = model(input_ids=batch_input_ids, labels=batch_labels)
        logits = outputs.logits

    # Calculate loss
    loss = outputs.loss
    total_eval_loss += loss.item()

    # Compute predictions and ground-truth labels
    predictions = torch.argmax(logits, dim=1).flatten()
    eval_predictions.extend(predictions.cpu().numpy())
    eval_labels.extend(batch_labels.cpu().numpy())

    # Calculate accuracy for this batch
    accuracy = (predictions == batch_labels).cpu().numpy().mean()
    total_eval_accuracy += accuracy

    nb_eval_steps += 1

# Calculate average evaluation metrics
avg_eval_loss = total_eval_loss / nb_eval_steps
avg_eval_accuracy = total_eval_accuracy / nb_eval_steps

# Compute additional evaluation metrics (e.g., precision, recall, F1-score)
eval_precision = precision_score(eval_labels, eval_predictions)
eval_recall = recall_score(eval_labels, eval_predictions)
eval_f1 = f1_score(eval_labels, eval_predictions)

# Print evaluation metrics
print(f'Validation Loss: {avg_eval_loss:.4f}')
print(f'Validation Accuracy: {avg_eval_accuracy:.4f}')
print(f'Validation Precision: {eval_precision:.4f}')
print(f'Validation Recall: {eval_recall:.4f}')
print(f'Validation F1-score: {eval_f1:.4f}')


Validation Loss: 0.7223
Validation Accuracy: 0.5714
Validation Precision: 0.5714
Validation Recall: 1.0000
Validation F1-score: 0.7273


## Testing with new dataset

In [ ]:
import torch

# New sentence to classify
new_sentence = "মই তেওঁলৈ পথ প্ৰদান কৰিলোঁ।"  #I gave him the way.

# Preprocess the new sentence
inputs = tokenizer(new_sentence, return_tensors="pt", padding=True, truncation=True)
inputs.to(device)

# Pass the preprocessed input to the model for inference
with torch.no_grad():
    outputs = model(**inputs)

# Interpret the model's output to obtain predictions
predicted_class_index = torch.argmax(outputs.logits, dim=1).item()

# Print the predicted class label
print("Predicted class label:", predicted_class_index)


Predicted class label: 1


In [ ]:

import torch

# New sentence to classify
new_sentence = "মই তেওঁলৈ পথ প্ৰদান কৰিলোঁ।"  #I provided him with directions.

# Preprocess the new sentence
inputs = tokenizer(new_sentence, return_tensors="pt", padding=True, truncation=True)
inputs.to(device)

# Pass the preprocessed input to the model for inference
with torch.no_grad():
    outputs = model(**inputs)

# Interpret the model's output to obtain predictions
predicted_class_index = torch.argmax(outputs.logits, dim=1).item()

# Print the predicted class label
print("Predicted class label:", predicted_class_index)


Predicted class label: 1


In [ ]:
import torch

# New sentence to classify
new_sentence = "মই তেওঁলৈ পথ প্ৰদান কৰিলোঁ।"  #I gave him the way.

# Preprocess the new sentence
inputs = tokenizer(new_sentence, return_tensors="pt", padding=True, truncation=True)
inputs.to(device)

# Pass the preprocessed input to the model for inference
with torch.no_grad():
    outputs = model(**inputs)

# Interpret the model's output to obtain predictions
predicted_class_index = torch.argmax(outputs.logits, dim=1).item()

# Print the predicted class label
print("Predicted class label:", predicted_class_index)


Predicted class label: 1


In [ ]:
import torch

# New sentence to classify
new_sentence = "সেই লোক বৰুষ ধৰ্মবিৰোধী।"

# Preprocess the new sentence
inputs = tokenizer(new_sentence, return_tensors="pt", padding=True, truncation=True)
inputs.to(device)

# Pass the preprocessed input to the model for inference
with torch.no_grad():
    outputs = model(**inputs)

# Interpret the model's output to obtain predictions
predicted_class_index = torch.argmax(outputs.logits, dim=1).item()

# Print the predicted class label
print("Predicted class label:", predicted_class_index)


Predicted class label: 1


In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from keras.preprocessing.sequence import pad_sequences  # Import pad_sequences from Keras

# Load the dataset
#df = pd.read_csv("assamese_hate_speech_dataset.csv")

# Tokenize the text data using the Assamese tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
tokenized_texts = [tokenizer.tokenize(text) for text in df['Text']]

# Set maximum sequence length
max_len = 128

# Pad sequences to maximum length using Keras pad_sequences
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(tokens) for tokens in tokenized_texts],
                          maxlen=max_len, dtype="long", truncating="post", padding="post")

# Create attention masks
attention_masks = [[float(i > 0) for i in seq] for seq in input_ids]

# Convert inputs and labels to PyTorch tensors
inputs = torch.tensor(input_ids)
masks = torch.tensor(attention_masks)
labels = torch.tensor(df['label'].values)

# Split the dataset into train and test sets
train_inputs, test_inputs, train_masks, test_masks, train_labels, test_labels = train_test_split(inputs, masks, labels, test_size=0.2, random_state=42)

# Create TensorDatasets
train_dataset = TensorDataset(train_inputs, train_masks, train_labels)
test_dataset = TensorDataset(test_inputs, test_masks, test_labels)

# Define batch size and create DataLoaders
batch_size = 32
train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset), batch_size=batch_size)

# Load pre-trained BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)

# Define optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)

# Training loop
epochs = 4
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.train()

for epoch in range(epochs):
    for batch in train_dataloader:
        batch_input_ids, batch_masks, batch_labels = batch
        batch_input_ids, batch_masks, batch_labels = batch_input_ids.to(device), batch_masks.to(device), batch_labels.to(device)
        model.zero_grad()
        outputs = model(input_ids=batch_input_ids, attention_mask=batch_masks, labels=batch_labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Model evaluation
model.eval()
total_eval_loss = 0
eval_predictions = []
eval_labels = []

for batch_input_ids, batch_masks, batch_labels in test_dataloader:
    batch_input_ids, batch_masks, batch_labels = batch_input_ids.to(device), batch_masks.to(device), batch_labels.to(device)
    with torch.no_grad():
        outputs = model(input_ids=batch_input_ids, attention_mask=batch_masks, labels=batch_labels)
        logits = outputs.logits
    loss = outputs.loss
    total_eval_loss += loss.item()
    predictions = torch.argmax(logits, dim=1).flatten()
    eval_predictions.extend(predictions.cpu().numpy())
    eval_labels.extend(batch_labels.cpu().numpy())

# Compute evaluation metrics
avg_eval_loss = total_eval_loss / len(test_dataloader)
eval_precision = precision_score(eval_labels, eval_predictions)
eval_recall = recall_score(eval_labels, eval_predictions)
eval_f1 = f1_score(eval_labels, eval_predictions)

# Print evaluation metrics
print(f'Validation Loss: {avg_eval_loss:.4f}')
print(f'Validation Precision: {eval_precision:.4f}')
print(f'Validation Recall: {eval_recall:.4f}')
print(f'Validation F1-score: {eval_f1:.4f}')


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Validation Loss: 0.5874
Validation Precision: 0.7407
Validation Recall: 1.0000
Validation F1-score: 0.8511
